In [99]:
from sdim import *
import cirq # using cirq to visualize the circuit
import numpy as np

### Encoding Steane Code
This notebook follows the work of [MITRE](https://stem.mitre.org/quantum/error-correction-codes/steane-ecc.html) for implementing Steane's Error Correcting Code

The circuit being read prepares the logical $\ket{0}$ codeword

In [100]:
encoding_circuit = Circuit(7, 2)
encoding_circuit.add_gate("H", [4, 5,6])
encoding_circuit.add_gate("CNOT", 0, [1, 2])
encoding_circuit.add_gate("CNOT", 6, [0, 1, 3])
encoding_circuit.add_gate("CNOT", 5, [0, 2, 3])
encoding_circuit.add_gate("CNOT", 4, [1, 2, 3])

Circuit(num_qudits=7, dimension=2, operations=[CircuitInstruction(gate_data=GateData(gateMap={'I': Gate(name='I', arg_count=1, gate_id=0), 'X': Gate(name='X', arg_count=1, gate_id=1), 'X_INV': Gate(name='X_INV', arg_count=1, gate_id=2), 'Z': Gate(name='Z', arg_count=1, gate_id=3), 'Z_INV': Gate(name='Z_INV', arg_count=1, gate_id=4), 'H': Gate(name='H', arg_count=1, gate_id=5), 'H_INV': Gate(name='H_INV', arg_count=1, gate_id=6), 'P': Gate(name='P', arg_count=1, gate_id=7), 'P_INV': Gate(name='P_INV', arg_count=1, gate_id=8), 'CNOT': Gate(name='CNOT', arg_count=2, gate_id=9), 'CNOT_INV': Gate(name='CNOT_INV', arg_count=2, gate_id=10), 'CZ': Gate(name='CZ', arg_count=2, gate_id=11), 'CZ_INV': Gate(name='CZ_INV', arg_count=2, gate_id=12), 'SWAP': Gate(name='SWAP', arg_count=2, gate_id=13), 'M': Gate(name='M', arg_count=1, gate_id=14), 'M_X': Gate(name='M_X', arg_count=1, gate_id=15)}, aliasMap={'R': 'H', 'DFT': 'H', 'R_INV': 'H_INV', 'DFT_INV': 'H_INV', 'H_DAG': 'H_INV', 'R_DAG': 'H_INV',

### Decoding Circuit
This circuit reads out the bit index for a bitflip (X) and phase flip (Z) error. It is technically possible to use only 3 ancilla instead of 6, opting to reset the measurement qubits and storing the outcome.

In [101]:
bitflip_detection = Circuit(13, 2)
bitflip_detection.add_gate("CNOT", [0, 2, 4, 6], 7)
bitflip_detection.add_gate("CNOT", [1, 2, 5, 6], 8)
bitflip_detection.add_gate("CNOT", [3, 4, 5, 6], 9)
bitflip_detection.add_gate("M", [7, 8, 9])

Circuit(num_qudits=13, dimension=2, operations=[CircuitInstruction(gate_data=GateData(gateMap={'I': Gate(name='I', arg_count=1, gate_id=0), 'X': Gate(name='X', arg_count=1, gate_id=1), 'X_INV': Gate(name='X_INV', arg_count=1, gate_id=2), 'Z': Gate(name='Z', arg_count=1, gate_id=3), 'Z_INV': Gate(name='Z_INV', arg_count=1, gate_id=4), 'H': Gate(name='H', arg_count=1, gate_id=5), 'H_INV': Gate(name='H_INV', arg_count=1, gate_id=6), 'P': Gate(name='P', arg_count=1, gate_id=7), 'P_INV': Gate(name='P_INV', arg_count=1, gate_id=8), 'CNOT': Gate(name='CNOT', arg_count=2, gate_id=9), 'CNOT_INV': Gate(name='CNOT_INV', arg_count=2, gate_id=10), 'CZ': Gate(name='CZ', arg_count=2, gate_id=11), 'CZ_INV': Gate(name='CZ_INV', arg_count=2, gate_id=12), 'SWAP': Gate(name='SWAP', arg_count=2, gate_id=13), 'M': Gate(name='M', arg_count=1, gate_id=14), 'M_X': Gate(name='M_X', arg_count=1, gate_id=15)}, aliasMap={'R': 'H', 'DFT': 'H', 'R_INV': 'H_INV', 'DFT_INV': 'H_INV', 'H_DAG': 'H_INV', 'R_DAG': 'H_INV'

In [102]:
phaseflip_detection = Circuit(13,2)
phaseflip_detection.add_gate("H", [10, 11, 12])
phaseflip_detection.add_gate("CNOT", 10, [0, 2, 4, 6])
phaseflip_detection.add_gate("CNOT", 11, [1, 2, 5, 6])
phaseflip_detection.add_gate("CNOT", 12, [3, 4, 5, 6])
phaseflip_detection.add_gate("H", [10, 11, 12])
phaseflip_detection.add_gate("M", [10, 11, 12])

# Create the full decoding circuit
decoding_circuit = bitflip_detection + phaseflip_detection

cirq_circuit = circuit_to_cirq_circuit(decoding_circuit, print_circuit=True)

             ┌──────────────────────────────────────────┐   ┌────────────────────────────────────────────────────────┐   ┌────────────────────────────┐   ┌────────────────────────────────────────────────────────┐   ┌───────────────────────────────────────────────────────────┐   ┌─────────────────────────────────────────────┐   ┌───────────────────────────────┐   ┌─────────────────┐   ┌─────────────────┐
0 (d=2): ─────CNOT_2_control───────────────────────────────────────────────CNOT_2_target───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
              │                                                            │
1 (d=2): ─────┼─────────────CNOT_2_control─────────────────────────────────┼─────────────CNOT_2_targ

### Error insertion
The Steane error correcting code has distance 3 and corrects up to a single arbitrary error. There is no need to have the error be on the same qubit, but it must not be more than 1 bit and 1 phase flip in total.

In [103]:
import random
# Generate a random error
qubit_index = random.randint(0, 6)
error_type = random.choice(["X", "Y", "Z"])
probability = 0.8
error_occured = True if np.random.rand() < probability else False
if error_type == "X" and error_occured:
    encoding_circuit.add_gate("X", qubit_index)
if error_type == "Z" and error_occured:
    encoding_circuit.add_gate("Z", qubit_index)
if error_type == "Y" and error_occured:
    encoding_circuit.add_gate("X", qubit_index)
    encoding_circuit.add_gate("Z", qubit_index)
if error_occured:
    print("Error type:", error_type, " on qubit", qubit_index)
else:
    print("No error occured")

Error type: Z  on qubit 1


### Syndrome Measurement

In [104]:
final_circuit = encoding_circuit + decoding_circuit
program = Program(final_circuit)
results = program.simulate(show_measurement=True)

Measurement results:
Measured qudit (7) as (0) and was deterministic
Measured qudit (8) as (0) and was deterministic
Measured qudit (9) as (0) and was deterministic
Measured qudit (10) as (0) and was deterministic
Measured qudit (11) as (1) and was deterministic
Measured qudit (12) as (0) and was deterministic


### Syndrome Lookup Table

| Broken Qubit Index | $bit_0$ Value | $bit_1$ Value | $bit_2$ Value |
|--------------------|---------------|---------------|---------------|
| None               | 0             | 0             | 0             |
| 0                  | 0             | 1             | 1             |
| 1                  | 0             | 1             | 1             |
| 2                  | 1             | 1             | 1             |
| 3                  | 1             | 0             | 0             |
| 4                  | 1             | 0             | 1             |
| 5                  | 1             | 1             | 0             |
| 6                  | 1             | 1             | 1             |

Worth noting is that the index value is offset by one to include the occurence of no error.

The bits are also reverse order to the output one, with qudit (7) and qudit (10) representing $bit_2$ and qudit (9) and qudit (12) representing $bit_0$


In [105]:
write_circuit(final_circuit, "css_steane_final.chp", comment="Error type {} on qubit {}".format(error_type, qubit_index))

'/home/stevenn/projects/sdim/src/sdim/../circuits/css_steane_final.chp'